In [1]:
import json
import os

In [2]:
def get_prompts(filename):
    with open(filename) as f:
        prompts = []
        for line in f:
            prompts.append(json.loads(line))
    return prompts

In [3]:
benchmark_root = "../Quality_Analyzer/Quality_Filtered_Suggestions/"
dir_list = os.listdir(benchmark_root)

In [4]:
repair_root ="../Repair_Benchmarks/"

In [5]:
total_prompts = 0
for file in dir_list:
    if ".DS_Store" in file:
        continue
    
    prompt_list = []
    print("Processing file: ", file)
    benchmark_path = os.path.join(benchmark_root, file)
    prompts = get_prompts(benchmark_path)
    print("Number of prompts: ", len(prompts))
    dataset_type = "Java"
    if 'python' in file.lower():
        dataset_type = "Python"
    for prompt in prompts:
        # print("Prompt: ", prompt["task_id"])
        for i in range(len(prompt["suggestions"])):
            suggestion = prompt["suggestions"][i]
            if suggestion["Is_Vulnerable"]:
                new_prompt = prompt.copy()
                new_prompt.pop("suggestions", None)
                new_prompt["task_id"] = str(prompt["task_id"])+"_"+str(i)
                new_prompt["old_prompt"] = suggestion["fixed_generated_text"]
                new_prompt["repair_prompt"] = suggestion["fixed_generated_text"]
                prompt_text = ""
                if dataset_type == "Python":
                    for j in range(len(suggestion["Analyzer_Result"])):
                        prompt_text += "\n# Fix: At line " + str(suggestion["Analyzer_Result"][j]["line_number"]) + ", " + suggestion["Analyzer_Result"][j]["issue_text"]
                        # print(prompt_text)
                else:
                    if type(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]) is list:
                        for j in range(len(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"])):
                            
                            if type(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]) is list:
                                    for k in range(len(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"])):
                                        if "@start" in suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"][k]:
                                            prompt_text += "\n// Fix: At line " + str(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"][k]["@start"]) + ", " + suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                            # print(prompt_text)
                                        else:
                                            prompt_text += "\n// Fix: "+ suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                            # print(prompt_text)
                            else:
                                if "@start" in suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]:
                                    prompt_text += "\n// Fix: At line " + str(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["SourceLine"]["@start"]) + ", " + suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                    # print(prompt_text)
                                else:
                                    prompt_text += "\n// Fix: "+ suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"][j]["ShortMessage"]
                                    # print(prompt_text)
                    else:
                        if type(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]) is list:
                            for j in range(len(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"])):
                                if "@start" in suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"][j]:
                                    prompt_text += "\n// Fix: At line " + str(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"][j]["@start"]) + ", " + suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                    # print(prompt_text)
                                else:
                                    prompt_text += "\n// Fix: "+ suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                    # print(prompt_text)
                        else:
                            if "@start" in suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]:
                                prompt_text += "\n// Fix: At line " + str(suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["SourceLine"]["@start"]) + ", " + suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                # print(prompt_text)
                            else:
                                prompt_text += "\n// Fix: "+ suggestion["Analyzer_Result"]["BugCollection"]["BugInstance"]["ShortMessage"]
                                # print(prompt_text)
                new_prompt["repair_prompt"] += prompt_text+"\n"
                prompt_list.append(new_prompt)

    print("Number of new prompts: ", len(prompt_list))
    total_prompts += len(prompt_list)
    repair_path = os.path.join(repair_root, file)
    with open(repair_path, 'w') as f:
        for prompt in prompt_list:
            f.write(json.dumps(prompt))
            f.write("\n")
print("Total prompts: ", total_prompts)

Processing file:  SecurityEval_python_codeparrot-small_128_10.jsonl
Number of prompts:  121
Number of new prompts:  222
Processing file:  SOEvalJava_PolyCoder-0.4B_128_10.jsonl
Number of prompts:  28
Number of new prompts:  16
Processing file:  CoderEval4Python_prompt_codeparrot_128_10.jsonl
Number of prompts:  230
Number of new prompts:  31
Processing file:  SOEvalPython_codegen-350M-mono_128_10.jsonl
Number of prompts:  42
Number of new prompts:  28
Processing file:  HumanEval_java_incoder-1B_128_10.jsonl
Number of prompts:  161
Number of new prompts:  18
Processing file:  SecurityEval_python_codegen-2B-multi_128_10.jsonl
Number of prompts:  121
Number of new prompts:  253
Processing file:  HumanEval_python_PolyCoder-2.7B_128_10.jsonl
Number of prompts:  164
Number of new prompts:  34
Processing file:  SOEvalJava_codegen-2B-multi_128_10.jsonl
Number of prompts:  28
Number of new prompts:  42
Processing file:  CoderEval4Python_prompt_codegen-350M-multi_128_10.jsonl
Number of prompts: 